In [ ]:
#!pip install simple_salesforce #allows us to work with salesforce API

In [1]:
import os
orig_working_directory=os.getcwd()
orig_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW\\ETL_Project'

In [2]:
os.chdir(os.path.join('..'))
curr_working_directory=os.getcwd()
curr_working_directory

'C:\\Users\\clemi\\Desktop\\Clementine-GWU-HW'

## Import Dependencies

In [3]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

## Import configuration variables

In [4]:
from salesforce_config import sf_username, sf_password, sf_security_token
from salesforce_config import remote_db_endpoint, remote_db_port
from salesforce_config import remote_db_name, remote_db_user, remote_db_pwd

## Set up simple_salesforce

In [5]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [6]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [7]:
student_data = pd.read_sql("SELECT * FROM student", conn)
student_data.head(5)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


## Transform Data

In [8]:
#student_data['FullName'] = student_data['FirstName'].str.cat(student_data['LastName'], sep=" ")
student_data=student_data.rename(columns={"ID_Student":"ID_Student__c","StudentID":"StudentID__c","LastName":"Name",
                                       "FirstName":"FirstName__c","MiddleName":"MiddleName__c","Gender":"Gender__c",
                                        "BirthDate":"BirthDate__c"})
student_data.head(5)

,ID_Student__c,StudentID__c,Name,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


## Convert DataFrame to a list of Dictionaries

In [ ]:
student_load = student_data.to_dict('records')
student_load

## Inserting rows in bulk

In [ ]:
try:
    sf.bulk.Student__c.insert(student_load) #Student__c is the API name
except Exception as e:
    print(e)